# Explore and Cluster the Neighborhoods in Toronto

## Import Modules

In [1]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    altair-3.2.0               |           py36_0         770 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.3 MB

The following NEW packages will be 

## Recreate merged dataframe with longitude and latitude 

In [2]:
import pandas as pd 
df_coord = pd.read_csv('https://cocl.us/Geospatial_data')
df_coord = df_coord.rename(columns={'Postal Code': 'Postcode'}) 

df_coord.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [3]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
tables = pd.read_html(url)
df=tables[0]

borough_mask = df.index[df['Borough'] == 'Not assigned']
neighborhood_mask = df.index[df['Neighbourhood'] == 'Not assigned']
neighborhood_and_borough_mask = borough_mask & neighborhood_mask

df.drop(df.index[borough_mask], inplace=True)
df.reset_index(drop=True, inplace=True)

neighborhood_mask = df.index[df['Neighbourhood'] == 'Not assigned']
for idx in neighborhood_mask:
    df['Neighbourhood'][idx] = df['Borough'][idx]
    

df2 = df.groupby(['Postcode','Borough'], sort=False).agg( ', '.join)
df_grouped=df2.reset_index()

df_grouped.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [4]:
df_final = df_grouped.groupby(['Postcode','Borough'], sort=False).agg(lambda x: ','.join(x))
df_final.reset_index(level=['Postcode','Borough'], inplace=True)
print(df_final.shape)
df_final.head()

(103, 3)


,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [5]:
df_toronto = pd.merge(df_final, df_coord, on='Postcode')
print(df_toronto.shape)
df_toronto.head(10)

(103, 5)


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


## Use Geopy to get latitudes and longitudes of Toronto, Canada

In [6]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="Toronto")
location = geolocator.geocode(address)
latitude_toronto = location.latitude
longitude_toronto = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude_toronto, longitude_toronto))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [7]:
map_toronto = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=10)

# add markers to map
for lat, lng, borough, Neighbourhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighbourhood']):
    label = '{}, {}'.format(Neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Foursquare Developer Credentials

In [8]:
CLIENT_ID = '4A3M4FLKYYCPFHOXG5KZ2ITDXJFA4E1R0G3JP0VAK2UCM0TK' # your Foursquare ID
CLIENT_SECRET = 'ZCBTEZ2SX1RQMTIOKGSQWKA2TQWK4X4Z322YGSP1F12D0WCR' # your Foursquare Secret
VERSION = '20191018' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 4A3M4FLKYYCPFHOXG5KZ2ITDXJFA4E1R0G3JP0VAK2UCM0TK
CLIENT_SECRET:ZCBTEZ2SX1RQMTIOKGSQWKA2TQWK4X4Z322YGSP1F12D0WCR


In [9]:
# defining radius and limit of venues to get
radius=500
LIMIT=100

In [10]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [11]:
import requests

venues_toronto = getNearbyVenues(names=df_toronto['Neighbourhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )

Parkwoods
Victoria Village
Harbourfront, Regent Park
Lawrence Heights, Lawrence Manor
Queen's Park
Islington Avenue
Rouge, Malvern
Don Mills North
Woodbine Gardens, Parkview Hill
Ryerson, Garden District
Glencairn
Cloverdale, Islington, Martin Grove, Princess Gardens, West Deane Park
Highland Creek, Rouge Hill, Port Union
Flemingdon Park, Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Downsview North, Wilson Heights
Thorncliffe Park
Adelaide, King, Richmond
Dovercourt Village, Dufferin
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto
Harbourfront East, Toronto Islands, Union Station
Little Portugal, Trinity
East Birchmount Park, Ionview, Kennedy Park
Bayview Village
CFB Toronto, Downsview East
The D

In [12]:
print(venues_toronto.shape)
venues_toronto.head(10)

(2272, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
5,Victoria Village,43.725882,-79.315572,The Frig,43.727051,-79.317418,French Restaurant
6,Victoria Village,43.725882,-79.315572,Eglinton Ave E & Sloane Ave/Bermondsey Rd,43.726086,-79.313620,Intersection
7,Victoria Village,43.725882,-79.315572,Pizza Nova,43.725824,-79.312860,Pizza Place
8,"Harbourfront, Regent Park",43.654260,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
9,"Harbourfront, Regent Park",43.654260,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop


In [13]:
venues_toronto.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,6,6,6,6,6,6
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",2,2,2,2,2,2
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",11,11,11,11,11,11
"Alderwood, Long Branch",11,11,11,11,11,11
"Bathurst Manor, Downsview North, Wilson Heights",20,20,20,20,20,20
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",23,23,23,23,23,23
Berczy Park,55,55,55,55,55,55


## Analyze Neighborhoods 

In [14]:
# one hot encoding
onehot_toronto = pd.get_dummies(venues_toronto[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot_toronto['Neighbourhood'] = venues_toronto['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [onehot_toronto.columns[-1]] + list(onehot_toronto.columns[:-1])
onehot_toronto.head()

,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Neighbourhood
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Parkwoods
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Parkwoods
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Victoria Village
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Victoria Village
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Victoria Village


In [15]:
print(onehot_toronto.shape)

(2272, 276)


### Group rows by neighborhood and by the mean of the frequency of occurrence of each category

In [16]:
grouped_toronto = onehot_toronto.groupby('Neighbourhood').mean().reset_index()
print(grouped_toronto.shape)
grouped_toronto


(100, 276)


,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,...,0.0,0.010000,0.000000,0.00,0.000000,0.0000,0.010000,0.000000,0.01,0.000000
1,Agincourt,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.00,0.000000,0.0000,0.000000,0.000000,0.00,0.000000
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.00,0.000000,0.0000,0.000000,0.000000,0.00,0.000000
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.00,0.000000,0.0000,0.000000,0.000000,0.00,0.000000
4,"Alderwood, Long Branch",0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.00,0.000000,0.0000,0.000000,0.000000,0.00,0.000000
5,"Bathurst Manor, Downsview North, Wilson Heights",0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.05,0.000000,0.0000,0.000000,0.000000,0.00,0.000000
6,Bayview Village,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.00,0.000000,0.0000,0.000000,0.000000,0.00,0.000000
7,"Bedford Park, Lawrence Manor East",0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.043478,...,0.0,0.000000,0.000000,0.00,0.000000,0.0000,0.000000,0.000000,0.00,0.000000
8,Berczy Park,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.018182,0.000000,0.00,0.000000,0.0000,0.000000,0.000000,0.00,0.000000
9,"Birch Cliff, Cliffside West",0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.00,0.000000,0.0000,0.000000,0.000000,0.00,0.000000


### Top 5 Most Common Venues of each Neighborhood

In [20]:
top_venues = 5

for hood in grouped_toronto['Neighbourhood']:
    print("----"+hood+"----")
    temp = grouped_toronto[grouped_toronto['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(top_venues))
    print('\n')

----Adelaide, King, Richmond----
              venue  freq
0       Coffee Shop  0.07
1              Café  0.05
2               Bar  0.04
3   Thai Restaurant  0.04
4  Asian Restaurant  0.03


----Agincourt----
                venue  freq
0      Breakfast Spot  0.17
1          Print Shop  0.17
2              Lounge  0.17
3  Chinese Restaurant  0.17
4      Sandwich Place  0.17


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                       venue  freq
0                 Playground   0.5
1                       Park   0.5
2  Middle Eastern Restaurant   0.0
3                      Motel   0.0
4        Monument / Landmark   0.0


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
           venue  freq
0  Grocery Store  0.18
1       Pharmacy  0.09
2    Pizza Place  0.09
3   Liquor Store  0.09
4     Beer Store  0.09


----Alderwood, Long Branch----
                venue  freq
0         Pizza Place  0.18

### Make into Dataframe

Sort venues in descending order

In [21]:
def return_most_common_venues(row, top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:top_venues]

Create dataframe with top 10 venues

In [22]:
import numpy as np
top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
top10venues = pd.DataFrame(columns=columns)
top10venues['Neighbourhood'] = grouped_toronto['Neighbourhood']

for ind in np.arange(grouped_toronto.shape[0]):
    top10venues.iloc[ind, 1:] = return_most_common_venues(grouped_toronto.iloc[ind, :], top_venues)

top10venues.head(10)

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Thai Restaurant,Bar,Steakhouse,Bakery,Breakfast Spot,Restaurant,Sushi Restaurant,Asian Restaurant
1,Agincourt,Chinese Restaurant,Lounge,Sandwich Place,Breakfast Spot,Clothing Store,Print Shop,Yoga Studio,Discount Store,Dog Run,Doner Restaurant
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Playground,Park,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Pharmacy,Fried Chicken Joint,Pizza Place,Sandwich Place,Discount Store,Fast Food Restaurant,Beer Store,Japanese Restaurant,Liquor Store
4,"Alderwood, Long Branch",Pizza Place,Pharmacy,Athletics & Sports,Dance Studio,Coffee Shop,Pool,Pub,Sandwich Place,Skating Rink,Gym
5,"Bathurst Manor, Downsview North, Wilson Heights",Coffee Shop,Pet Store,Fast Food Restaurant,Sushi Restaurant,Middle Eastern Restaurant,Restaurant,Deli / Bodega,Fried Chicken Joint,Frozen Yogurt Shop,Diner
6,Bayview Village,Café,Japanese Restaurant,Bank,Chinese Restaurant,Yoga Studio,Drugstore,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
7,"Bedford Park, Lawrence Manor East",Coffee Shop,Italian Restaurant,Juice Bar,Grocery Store,Butcher,Spa,Fast Food Restaurant,Liquor Store,Sandwich Place,Restaurant
8,Berczy Park,Coffee Shop,Cocktail Bar,Italian Restaurant,Cheese Shop,Bakery,Beer Bar,Seafood Restaurant,Farmers Market,Café,Steakhouse
9,"Birch Cliff, Cliffside West",College Stadium,General Entertainment,Skating Rink,Café,Comic Shop,Dim Sum Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store


## Cluster Neighborhoods 
k-means into 5 clusters

In [23]:
# set number of clusters
kclusters = 5

grouped_clustering = grouped_toronto.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_ 
# to change use .astype() 

array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 4, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 3, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1], dtype=int32)

#### New Top 10 Venues Dataframe w/ Cluster

In [24]:
# add clustering labels
top10venues.insert(0, 'cluster_Labels', kmeans.labels_)

toronto_merged = df_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(top10venues.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head(10) # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,1.0,Food & Drink Shop,Park,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Yoga Studio
1,M4A,North York,Victoria Village,43.725882,-79.315572,0.0,Pizza Place,French Restaurant,Coffee Shop,Portuguese Restaurant,Hockey Arena,Intersection,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,0.0,Coffee Shop,Park,Bakery,Café,Theater,Breakfast Spot,Pub,Mexican Restaurant,Yoga Studio,Brewery
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763,0.0,Clothing Store,Furniture / Home Store,Coffee Shop,Miscellaneous Shop,Arts & Crafts Store,Accessories Store,Vietnamese Restaurant,Event Space,Boutique,Women's Store
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494,0.0,Coffee Shop,Gym,Park,Burger Joint,Diner,Yoga Studio,Café,Italian Restaurant,Seafood Restaurant,Sandwich Place
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,3.0,Fast Food Restaurant,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,College Rec Center
7,M3B,North York,Don Mills North,43.745906,-79.352188,0.0,Café,Japanese Restaurant,Gym / Fitness Center,Caribbean Restaurant,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937,0.0,Pizza Place,Fast Food Restaurant,Athletics & Sports,Gastropub,Intersection,Pet Store,Pharmacy,Bus Line,Bank,Gym / Fitness Center
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,0.0,Clothing Store,Coffee Shop,Cosmetics Shop,Café,Middle Eastern Restaurant,Ice Cream Shop,Pizza Place,Fast Food Restaurant,Sporting Goods Shop,Italian Restaurant


Drop rows with no data available

In [25]:
toronto_merged=toronto_merged.dropna()

In [26]:
toronto_merged['cluster_Labels'] = toronto_merged.cluster_Labels.astype(int)

In [27]:
# create map
map = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['cluster_Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map)
       
map

## Examine Clusters

### 1st Cluster

In [28]:
toronto_merged.loc[toronto_merged['cluster_Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,0,Pizza Place,French Restaurant,Coffee Shop,Portuguese Restaurant,Hockey Arena,Intersection,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store
2,Downtown Toronto,0,Coffee Shop,Park,Bakery,Café,Theater,Breakfast Spot,Pub,Mexican Restaurant,Yoga Studio,Brewery
3,North York,0,Clothing Store,Furniture / Home Store,Coffee Shop,Miscellaneous Shop,Arts & Crafts Store,Accessories Store,Vietnamese Restaurant,Event Space,Boutique,Women's Store
4,Queen's Park,0,Coffee Shop,Gym,Park,Burger Joint,Diner,Yoga Studio,Café,Italian Restaurant,Seafood Restaurant,Sandwich Place
7,North York,0,Café,Japanese Restaurant,Gym / Fitness Center,Caribbean Restaurant,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
8,East York,0,Pizza Place,Fast Food Restaurant,Athletics & Sports,Gastropub,Intersection,Pet Store,Pharmacy,Bus Line,Bank,Gym / Fitness Center
9,Downtown Toronto,0,Clothing Store,Coffee Shop,Cosmetics Shop,Café,Middle Eastern Restaurant,Ice Cream Shop,Pizza Place,Fast Food Restaurant,Sporting Goods Shop,Italian Restaurant
11,Etobicoke,0,Golf Course,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Yoga Studio,Dessert Shop
12,Scarborough,0,Bar,Yoga Studio,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Dim Sum Restaurant
13,North York,0,Gym,Asian Restaurant,Beer Store,Coffee Shop,Fast Food Restaurant,Bike Shop,Sporting Goods Shop,Sandwich Place,Supermarket,Discount Store


### 2nd Cluster

In [29]:
toronto_merged.loc[toronto_merged['cluster_Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,1,Food & Drink Shop,Park,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Yoga Studio
10,North York,1,Pub,Metro Station,Park,Japanese Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio
21,York,1,Park,Women's Store,Fast Food Restaurant,Market,Yoga Studio,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop
35,East York,1,Park,Coffee Shop,Convenience Store,Yoga Studio,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
40,North York,1,Construction & Landscaping,Playground,Airport,Park,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
49,North York,1,Park,Bakery,Construction & Landscaping,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
61,Central Toronto,1,Park,Bus Line,Swim School,Yoga Studio,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore
64,York,1,Park,Convenience Store,Yoga Studio,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
66,North York,1,Park,Bank,Convenience Store,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
68,Central Toronto,1,Trail,Park,Sushi Restaurant,Jewelry Store,Yoga Studio,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop


### 3rd Cluster

In [30]:
toronto_merged.loc[toronto_merged['cluster_Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
45,North York,2,Cafeteria,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Fast Food Restaurant


### 4th Cluster

In [31]:
toronto_merged.loc[toronto_merged['cluster_Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Scarborough,3,Fast Food Restaurant,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,College Rec Center


### 5th Cluster

In [32]:
toronto_merged.loc[toronto_merged['cluster_Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
57,North York,4,Baseball Field,Yoga Studio,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Dim Sum Restaurant
